In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats

In [29]:
at = pd.read_csv('train.csv')

In [6]:
at.nunique()

time        713
trt           4
age          59
wtkg        667
hemo          2
homo          2
drugs         2
karnof        4
oprior        2
z30           2
zprior        1
preanti     813
race          2
gender        2
str2          2
strat         3
symptom       2
treat         2
offtrt        2
cd40        484
cd420       570
cd80       1090
cd820      1050
Target        2
dtype: int64

In [8]:
at.isnull().sum()

time       0
trt        0
age        0
wtkg       0
hemo       0
homo       0
drugs      0
karnof     0
oprior     0
z30        0
zprior     0
preanti    0
race       0
gender     0
str2       0
strat      0
symptom    0
treat      0
offtrt     0
cd40       0
cd420      0
cd80       0
cd820      0
Target     0
dtype: int64

In [45]:
y = at['Target'].replace({'censoring': '0', 'failure':1}).astype('int')
x = at.drop('Target', axis = 1)

In [47]:
x.drop(['trt', 'hemo', 'homo', 'drugs', 'oprior', 'z30', 'zprior', 'race', 'gender', 'str2', 'strat', 'symptom', 'treat', 'offtrt'], axis =1, inplace = True)

In [46]:
x_enc = pd.get_dummies(x[['trt', 'hemo', 'homo', 'drugs', 'oprior', 'z30', 'zprior', 'race', 'gender', 'str2', 'strat', 'symptom', 'treat', 'offtrt']], drop_first = True, dtype = 'int')

In [48]:
x_f = pd.concat([x,x_enc], axis = 1)

In [49]:
x_f

,time,age,wtkg,karnof,preanti,cd40,cd420,cd80,cd820,trt_ZDV + ddI,trt_ZDV only,trt_ddI only,hemo_Yes,homo_Yes,drugs_Yes,oprior_Yes,z30_Yes,race_non-white,gender_Male,str2_Yes,strat_> 52 weeks,strat_Antiretroviral Naive,symptom_symp,treat_others,offtrt_Yes
0,948,48,89.8128,100,0,422,477,566,324,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,1002,61,49.4424,90,895,162,218,392,564,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0
2,961,45,88.4520,90,707,326,274,2063,1893,0,0,1,0,1,1,0,1,0,1,1,1,0,0,1,1
3,1166,47,85.2768,100,1399,287,394,1590,966,0,0,1,0,1,0,0,1,0,1,1,1,0,0,1,0
4,1090,43,66.6792,100,1352,504,353,870,782,0,1,0,0,1,0,0,1,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,1091,21,53.2980,100,842,152,109,561,720,0,0,1,1,0,0,0,1,0,1,1,1,0,0,1,1
2135,395,17,102.9672,100,417,373,218,1759,1030,0,1,0,1,0,0,0,1,1,1,1,1,0,0,0,1
2136,1104,53,69.8544,90,753,419,364,1391,1041,0,0,0,1,1,0,0,1,1,1,1,1,0,0,1,0
2137,465,14,60.0000,100,0,166,169,999,1838,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0


In [28]:
x

,time,age,wtkg,karnof,preanti,cd40,cd420,cd80,cd820
0,948,48,89.8128,100,0,422,477,566,324
1,1002,61,49.4424,90,895,162,218,392,564
2,961,45,88.4520,90,707,326,274,2063,1893
3,1166,47,85.2768,100,1399,287,394,1590,966
4,1090,43,66.6792,100,1352,504,353,870,782
...,...,...,...,...,...,...,...,...,...
2134,1091,21,53.2980,100,842,152,109,561,720
2135,395,17,102.9672,100,417,373,218,1759,1030
2136,1104,53,69.8544,90,753,419,364,1391,1041
2137,465,14,60.0000,100,0,166,169,999,1838


In [50]:
y.dtypes

dtype('int32')

In [51]:
from flaml import AutoML
automl = AutoML()
automl.fit(x_f,y, task = 'classification', metric = 'roc_auc', time_budget = 55)

[flaml.automl.logger: 08-05 09:44:21] {1680} INFO - task = classification
[flaml.automl.logger: 08-05 09:44:21] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 08-05 09:44:21] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 08-05 09:44:21] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 08-05 09:44:21] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 08-05 09:44:22] {2345} INFO - Estimated sufficient time budget=9931s. Estimated necessary time budget=229s.
[flaml.automl.logger: 08-05 09:44:22] {2392} INFO -  at 1.1s,	estimator lgbm's best error=0.1001,	best estimator lgbm's best error=0.1001
[flaml.automl.logger: 08-05 09:44:22] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 08-05 09:44:23] {2392} INFO -  at 2.0s,	estimator lgbm's best error=0.1001,	best estimator lgbm's best error=0.1001
[flaml.automl.logger: 08-05 09:44:

In [53]:
automl.best_estimator, automl.best_config

('lgbm',
 {'n_estimators': 56,
  'num_leaves': 12,
  'min_child_samples': 3,
  'learning_rate': 0.17402065726724145,
  'log_max_bin': 8,
  'colsample_bytree': 0.6649148062238498,
  'reg_alpha': 0.0009765625,
  'reg_lambda': 0.006761362450996487})

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y,automl.predict(x_f)))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1618
           1       0.97      0.92      0.95       521

    accuracy                           0.97      2139
   macro avg       0.97      0.96      0.96      2139
weighted avg       0.97      0.97      0.97      2139

